In [111]:
import boto3
import sagemaker
from pyathena import connect
import pandas as pd

## Data Wrangling

### Create and Verify Database

In [112]:
# Make athena db
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [6]:
ingest_create_athena_db_passed = False

In [113]:
# Set db name
database_name = "crops"

In [114]:
# Set S3 staging directory - temporary directory used for athena
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [115]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [116]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)
pd.read_sql(statement, conn)

CREATE DATABASE IF NOT EXISTS crops


""


In [117]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head()

,database_name
0,crops
1,default


In [118]:
if database_name in df_show.values:
    ingest_create_athena_db_passed = True

### Create Athena Tables from Local CSVs 
#### Crops

In [119]:
# Athena parameters
table_name_csv = "crops_long"

In [120]:
s3_private_path_csv = "s3://sagemaker-studio-998234604495-nyhifbo32oo/raw_data/"

In [121]:
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
        Area string,
        Item string,
        Element string,
        Unit string,
        variable string,
        value float
        
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name_csv, s3_private_path_csv
)

In [122]:
pd.read_sql(statement, conn)

""


In [123]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,tab_name
0,crops_long
1,crops_long_athena
2,crops_long_csv


In [124]:
if table_name_csv in df_show.values:
    ingest_create_athena_table_tsv_passed = True

In [125]:
item = "Camels"

statement = """SELECT * FROM {}.{}
    WHERE item = '{}' LIMIT 5""".format(
    database_name, table_name_csv, item
)

print(statement)

SELECT * FROM crops.crops_long
    WHERE item = 'Camels' LIMIT 5


In [126]:
camels = pd.read_sql(statement, conn)
camels.head()

,area,item,element,unit,variable,value
0,Burkina Faso,Camels,Stocks,Head,Y2017,19475.0
1,Chad,Camels,Stocks,Head,Y2017,7285309.0
2,China,Camels,Stocks,Head,Y2017,323270.0
3,Djibouti,Camels,Stocks,Head,Y2017,71060.0
4,Egypt,Camels,Stocks,Head,Y2017,155713.0
